In [1]:
import numpy as np
import librosa, librosa.display #librosa is a python package for music and audio analysis. It provides the building blocks necessary to create music information retrieval systems.
import matplotlib.pyplot as plt
class Preprocessing:
    def __init__(self):
        self.features = {}
        
    def insert_(self,key,value):
        self.features[key] = value
        
    def reset(self):
        self.features = {}
        
    def extract_features(self,file=""):
        signal, sample_rate = librosa.load(file, sr=22050)#plot signal
        self.insert_("signal",signal)
        self.insert_("sample rate",sample_rate)
        # FFT -> power spectrum
        # perform Fourier transform
        fft = np.fft.fft(signal)
        # calculate abs values on complex numbers to get magnitude
        spectrum = np.abs(fft)
        # create frequency variable
        f = np.linspace(0, sample_rate, len(spectrum))
        # take half of the spectrum and frequency
        left_spectrum = spectrum[:int(len(spectrum)/2)]#plot_fft
        left_f = f[:int(len(spectrum)/2)]#plot_fft
        self.insert_("FFT",spectrum)
        # STFT -> spectrogram
        hop_length = 512 # in num. of samples
        frame_length = 2*hop_length
        n_fft = 2048 # window in num. of samples
        # calculate duration hop length and window in seconds
        hop_length_duration = float(hop_length)/sample_rate
        n_fft_duration = float(n_fft)/sample_rate
        print("STFT hop length duration is: {}s".format(hop_length_duration))
        print("STFT window duration is: {}s".format(n_fft_duration))
        # perform stft
        stft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
        # calculate abs values on complex numbers to get magnitude
        spectrogram = np.abs(stft)#plot stft
        self.insert_("STFT",spectrogram)
        log_spectrogram = librosa.amplitude_to_db(spectrogram)
        self.insert_("STFT in Decibels",log_spectrogram)
        #ZCR
        n1=0
        n2=len(signal)
        #ZC Calculation
        zero_crossings = librosa.zero_crossings(signal[n1:n2], pad=False)
        self.insert_("Zero Crossings",sum(zero_crossings))
        #ZCR Calculation 
        zcrs = librosa.feature.zero_crossing_rate(signal[n1:n2],frame_length=frame_length,hop_length=hop_length)
        zcrs=list(np.around(np.array(zcrs)*frame_length).astype(int))
        self.insert_("Zero Crossing Rate",zcrs[0])
        # MFCCs
        # extract 13 MFCCs
        MFCCs = librosa.feature.mfcc(signal, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
        self.insert_("13 MFCCs",MFCCs)
        #Calculating Energy
        energy = np.array([sum(abs(signal[i:i+frame_length]**2)) for i in range(0, len(signal), hop_length)])
        self.insert_("Energy",energy)
        #Calculating RMSE
        rmse = librosa.feature.rms(signal, frame_length=frame_length, hop_length=hop_length, center=True)
        self.insert_("RMSE",rmse[0])
        return self.get_features()
        
    def get_features(self):
        return self.features
    
    def get_feature_row(self,path="",gender=""):
        self.extract_features(path)
        features=self.get_features()
        row={}
        count=1
        for i in features['13 MFCCs']:
            row[str(count)]=[i]
            count+=1
        row['Zero Crossing Rate']=[features['Zero Crossing Rate']]
        row['Energy']=[features['Energy']]
        row['RMSE']=[features['RMSE']]
        row['Gender']=[gender]
        return row
        
        
        
        
    

C:\Users\Christopher Gomes\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
C:\Users\Christopher Gomes\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\Christopher Gomes\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy 

In [2]:
prp = Preprocessing()

file = "audio\\Tracee Ellis Ross - A woman's fury holds lifetimes of wisdom _ TED.wav"
signal, sample_rate = librosa.load(file, sr=22050)#plot signal
signaln = 661794
gender = 'M'
df={'1':[], '2':[], '3':[], '4':[], '5':[], '6':[], '7':[], '8':[], '9':[], '10':[], '11':[], '12':[], '13':[], 'Zero Crossing Rate':[], 'Energy':[], 'RMSE':[], 'Gender':[]}
for i in range(signaln*2,len(signal),signaln):
    feature_row = prp.get_feature_row(file,gender)
    for key in feature_row.keys():
        print(key,feature_row[key])
        df[key]+=feature_row[key]
    prp.reset()
    

STFT hop length duration is: 0.023219954648526078s
STFT window duration is: 0.09287981859410431s
1 [array([-549.4721 , -549.4721 , -549.4721 , ..., -414.30707, -443.113  ,
       -478.66898], dtype=float32)]
2 [array([  0.     ,   0.     ,   0.     , ..., 105.20648,  91.96904,
        69.06843], dtype=float32)]
3 [array([ 0.       ,  0.       ,  0.       , ...,  3.492981 , 10.3196335,
       16.56613  ], dtype=float32)]
4 [array([ 0.        ,  0.        ,  0.        , ..., -0.47352785,
       -3.5120904 , -2.1720605 ], dtype=float32)]
5 [array([ 0.       ,  0.       ,  0.       , ..., 14.135945 , 11.297288 ,
        6.6030464], dtype=float32)]
6 [array([ 0.       ,  0.       ,  0.       , ...,  3.9533215, 12.513544 ,
       11.559525 ], dtype=float32)]
7 [array([0.       , 0.       , 0.       , ..., 2.6887238, 7.304452 ,
       9.204668 ], dtype=float32)]
8 [array([0.      , 0.      , 0.      , ..., 3.443011, 4.176102, 9.349329],
      dtype=float32)]
9 [array([ 0.       ,  0.       , 

       9.204668 ], dtype=float32)]
8 [array([0.      , 0.      , 0.      , ..., 3.443011, 4.176102, 9.349329],
      dtype=float32)]
9 [array([ 0.       ,  0.       ,  0.       , ...,  4.3933163,  5.89567  ,
       11.848382 ], dtype=float32)]
10 [array([ 0.      ,  0.      ,  0.      , ...,  6.730952,  8.099932,
       12.305925], dtype=float32)]
11 [array([ 0.      ,  0.      ,  0.      , ...,  8.291267, 10.350353,
        8.35581 ], dtype=float32)]
12 [array([0.       , 0.       , 0.       , ..., 5.951935 , 5.118434 ,
       1.0004238], dtype=float32)]
13 [array([ 0.       ,  0.       ,  0.       , ..., -2.3853595, -4.6644692,
       -2.8804314], dtype=float32)]
Zero Crossing Rate [array([  0,  96, 355, ...,  68,  71,  46])]
Energy [array([5.40957044e-09, 2.68058676e-08, 7.30910476e-08, ...,
       4.76022149e-03, 1.52911053e-03, 1.92409183e-04])]
RMSE [array([0.0000000e+00, 2.2984307e-06, 5.1164056e-06, ..., 3.0988092e-03,
       2.1560737e-03, 1.2960082e-03], dtype=float32)]
Gende

       4.76022149e-03, 1.52911053e-03, 1.92409183e-04])]
RMSE [array([0.0000000e+00, 2.2984307e-06, 5.1164056e-06, ..., 3.0988092e-03,
       2.1560737e-03, 1.2960082e-03], dtype=float32)]
Gender ['M']
STFT hop length duration is: 0.023219954648526078s
STFT window duration is: 0.09287981859410431s
1 [array([-549.4721 , -549.4721 , -549.4721 , ..., -414.30707, -443.113  ,
       -478.66898], dtype=float32)]
2 [array([  0.     ,   0.     ,   0.     , ..., 105.20648,  91.96904,
        69.06843], dtype=float32)]
3 [array([ 0.       ,  0.       ,  0.       , ...,  3.492981 , 10.3196335,
       16.56613  ], dtype=float32)]
4 [array([ 0.        ,  0.        ,  0.        , ..., -0.47352785,
       -3.5120904 , -2.1720605 ], dtype=float32)]
5 [array([ 0.       ,  0.       ,  0.       , ..., 14.135945 , 11.297288 ,
        6.6030464], dtype=float32)]
6 [array([ 0.       ,  0.       ,  0.       , ...,  3.9533215, 12.513544 ,
       11.559525 ], dtype=float32)]
7 [array([0.       , 0.       , 0. 

        6.6030464], dtype=float32)]
6 [array([ 0.       ,  0.       ,  0.       , ...,  3.9533215, 12.513544 ,
       11.559525 ], dtype=float32)]
7 [array([0.       , 0.       , 0.       , ..., 2.6887238, 7.304452 ,
       9.204668 ], dtype=float32)]
8 [array([0.      , 0.      , 0.      , ..., 3.443011, 4.176102, 9.349329],
      dtype=float32)]
9 [array([ 0.       ,  0.       ,  0.       , ...,  4.3933163,  5.89567  ,
       11.848382 ], dtype=float32)]
10 [array([ 0.      ,  0.      ,  0.      , ...,  6.730952,  8.099932,
       12.305925], dtype=float32)]
11 [array([ 0.      ,  0.      ,  0.      , ...,  8.291267, 10.350353,
        8.35581 ], dtype=float32)]
12 [array([0.       , 0.       , 0.       , ..., 5.951935 , 5.118434 ,
       1.0004238], dtype=float32)]
13 [array([ 0.       ,  0.       ,  0.       , ..., -2.3853595, -4.6644692,
       -2.8804314], dtype=float32)]
Zero Crossing Rate [array([  0,  96, 355, ...,  68,  71,  46])]
Energy [array([5.40957044e-09, 2.68058676e-08

       -2.8804314], dtype=float32)]
Zero Crossing Rate [array([  0,  96, 355, ...,  68,  71,  46])]
Energy [array([5.40957044e-09, 2.68058676e-08, 7.30910476e-08, ...,
       4.76022149e-03, 1.52911053e-03, 1.92409183e-04])]
RMSE [array([0.0000000e+00, 2.2984307e-06, 5.1164056e-06, ..., 3.0988092e-03,
       2.1560737e-03, 1.2960082e-03], dtype=float32)]
Gender ['M']
STFT hop length duration is: 0.023219954648526078s
STFT window duration is: 0.09287981859410431s
1 [array([-549.4721 , -549.4721 , -549.4721 , ..., -414.30707, -443.113  ,
       -478.66898], dtype=float32)]
2 [array([  0.     ,   0.     ,   0.     , ..., 105.20648,  91.96904,
        69.06843], dtype=float32)]
3 [array([ 0.       ,  0.       ,  0.       , ...,  3.492981 , 10.3196335,
       16.56613  ], dtype=float32)]
4 [array([ 0.        ,  0.        ,  0.        , ..., -0.47352785,
       -3.5120904 , -2.1720605 ], dtype=float32)]
5 [array([ 0.       ,  0.       ,  0.       , ..., 14.135945 , 11.297288 ,
        6.6030

In [6]:
prp = Preprocessing()
d = prp.get_feature_row("audio\\blues.00000.wav")
print(d.keys())

STFT hop length duration is: 0.023219954648526078s
STFT window duration is: 0.09287981859410431s
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 'Zero Crossing Rate', 'Energy', 'RMSE', 'Gender'])


In [3]:
import pandas as pd
df = pd.DataFrame(df)

In [4]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,Zero Crossing Rate,Energy,RMSE,Gender
0,"[-549.4721, -549.4721, -549.4721, -549.4721, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.23266, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.34091, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.4686885,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.132629, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.993511...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.4504786...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9862862,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -38.956985...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.6674175,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -27.23317,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3049675...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -14.417941...","[0, 96, 355, 487, 455, 451, 437, 442, 575, 633...","[5.4095704371510955e-09, 2.680586755114437e-08...","[0.0, 2.2984307e-06, 5.1164056e-06, 8.448549e-...",M
1,"[-549.4721, -549.4721, -549.4721, -549.4721, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.23266, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.34091, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.4686885,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.132629, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.993511...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.4504786...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9862862,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -38.956985...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.6674175,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -27.23317,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3049675...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -14.417941...","[0, 96, 355, 487, 455, 451, 437, 442, 575, 633...","[5.4095704371510955e-09, 2.680586755114437e-08...","[0.0, 2.2984307e-06, 5.1164056e-06, 8.448549e-...",M
2,"[-549.4721, -549.4721, -549.4721, -549.4721, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.23266, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.34091, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.4686885,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.132629, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.993511...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.4504786...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9862862,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -38.956985...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.6674175,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -27.23317,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3049675...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -14.417941...","[0, 96, 355, 487, 455, 451, 437, 442, 575, 633...","[5.4095704371510955e-09, 2.680586755114437e-08...","[0.0, 2.2984307e-06, 5.1164056e-06, 8.448549e-...",M
3,"[-549.4721, -549.4721, -549.4721, -549.4721, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.23266, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.34091, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.4686885,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.132629, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.993511...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.4504786...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9862862,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -38.956985...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.6674175,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -27.23317,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3049675...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -14.417941...","[0, 96, 355, 487, 455, 451, 437, 442, 575, 633...","[5.4095704371510955e-09, 2.680586755114437e-08...","[0.0, 2.2984307e-06, 5.1164056e-06, 8.448549e-...",M
4,"[-549.4721, -549.4721, -549.4721, -549.4721, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.23266, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.34091, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.4686885,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.132629, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.993511...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.4504786...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9862862,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -38.956985...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.6674175,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -27.23317,...","[0.0, 0.0, 0